In [61]:
import os
import json

import numpy as np
import pandas as pd

print(os.path.abspath(os.path.curdir))

/Users/nicholsn/Repos/metasearch/crawler/transform


In [42]:
# Directories with extracted files.
extract_path = os.path.abspath('../extract')
all_dirs = os.walk(extract_path).next()[1]
dirs = [d for d in all_dirs if not d[0] == '.']
for i in dirs:
    if not os.path.exists(i):
        os.mkdir(i)        

In [79]:
def apply_mapping(mapping, csv_path):
    df = pd.DataFrame()
    csv = pd.read_csv(csv_path)
    for col, elem in mapping.iteritems():
        series = csv[col]
        if elem.get('type') == 'category':
            result = series.apply(lambda x: elem.get(str(x)))
        elif elem.get('type') == 'number':
            lambda x: True if x % 2 == 0 else False
            result = series.apply(lambda x: np.NaN if str(x) in elem.keys() else x)
        else:
            result = series
        df[elem.get('element')] = result
    return df

In [81]:
# Read a mapping.json file from each sub directory to process.
for i in dirs:
    mapping = dict()
    mapping_file = os.path.join('.', i, 'mapping.json')
    if os.path.exists(mapping_file):
        with open(mapping_file, 'rb') as fi:
            mapping.update(json.load(fi))
        for key in mapping.iterkeys():
            csv_path = os.path.join(extract_path, i, key)
            csv_mapping = mapping.get(key)
            xfm = apply_mapping(csv_mapping, csv_path)
            pheno = os.path.join('.', i, i + '_' + 'phenotype.csv')
            xfm.to_csv(pheno, index=False)